In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from pandas import DataFrame
load_dotenv();

In [2]:
haslanger_woman = """(i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination"""

In [13]:
template_1 = """Definition: {variable} is a(n) {term} iff {definition}.

Using the above definition, is {element} a(n) {term}? Answer 'True', 'False', or 'Unknown'.
Answer:"""

In [14]:
prompt_1 = PromptTemplate(
    input_variables=["variable", "term", "definition", "element"], 
    template=template_1,
)

In [15]:
print(prompt_1.format(variable="S", term="woman", definition=haslanger_woman, element="Catherine the Great"))

Definition: S is a(n) woman iff (i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination.

Using the above definition, is Catherine the Great a(n) woman? Answer 'True', 'False', or 'Unknown'.
Answer:


In [16]:
template_2 = """Definition: {variable} is a(n) {term} iff {definition}.

Using the above definition, is {element} a(n) {term}? Answer 'True' or 'False'.
Answer: {answer}

Explain your reasoning.
Explanation:"""

In [17]:
prompt_2 = PromptTemplate(
    input_variables=["variable", "term", "definition", "element", "answer"], 
    template=template_2,
)

In [18]:
print(prompt_2.format(variable="S", term="woman", definition=haslanger_woman, element="Catherine the Great", answer=False))

Definition: S is a(n) woman iff (i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination.

Using the above definition, is Catherine the Great a(n) woman? Answer 'True' or 'False'.
Answer: False

Explain your reasoning.
Explanation:


In [19]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.1)
in_extension_chain = LLMChain(llm=llm, prompt=prompt_1, output_key="answer")
explanation_chain = LLMChain(llm=llm, prompt=prompt_2, output_key="explanation")
overall_chain = SequentialChain(
    chains=[in_extension_chain, explanation_chain],
    input_variables=["variable", "term", "definition", "element"],
    output_variables=["answer", "explanation"],
)

In [21]:
overall_chain({"variable": "S", "term": "woman", "definition": haslanger_woman, "element": "Sally Haslanger"})

{'variable': 'S',
 'term': 'woman',
 'definition': '(i) S is regularly and for the most part observed or imagined to have \ncertain bodily features presumed to be evidence of a female’s biological role in reproduction; \n(ii) that S has these features marks S within the dominant ideology of S’s society as someone who \nought to occupy certain kinds of social position that are in fact subordinate (and so motivates \nand justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays \na role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, \nand S’s satisfying (i) and (ii) plays a role in that dimension of subordination',
 'element': 'Sally Haslanger',
 'answer': 'Unknown',
 'explanation': 'The question does not provide enough information about Sally Haslanger to determine whether she fits the given definition of a woman. We would need information about her biological features, her social position, and wh

In [11]:
collection = [
    "Rosa Parks",
    "Hillary Clinton",
    "Pamela Anderson",
    "Lebron James",
    "Sally Haslanger",
    "Nancy Pelosi",
    "Catwoman",
    "Taylor Swift",
    "Julia Roberts",
    "James Earl Carter",
    "Peter Bogdanovich",
    "Elinor Ostrom",
    "Catherine of Aragon",
    "Catherine the Great",
    "Joyce Carol Oates",
    "Sally Hemings",
    "Caitlyn Jenner",
    "Kamala Harris",
]

In [12]:
df = DataFrame(
    [
        
        [ 
            response["element"], 
            response["answer"].strip(), 
            response["explanation"].strip() 
        ]
        for response in [ 
            overall_chain(
                {
                    "variable": "S", 
                    "term": "woman", 
                    "definition": haslanger_woman, 
                    "element": element
                }
            ) 
            for element in collection
        ]  
        
    ],
    columns = [ 'element' , 'in_extension', 'explanation' ]
)
df.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})

,element,in_extension,explanation
0,Rosa Parks,True,"Rosa Parks was regularly observed to have the bodily features associated with a female's biological role in reproduction, marking her as a woman within the dominant ideology of her society. Furthermore, her social position was indeed subordinate and oppressive, as evidenced by her role in the Civil Rights Movement fighting against racial segregation. Therefore, she satisfies all the conditions in the definition of a woman."
1,Hillary Clinton,True,"Hillary Clinton is observed to have the bodily features associated with the female biological role in reproduction, marking her as female within the dominant ideology of her society. She has also occupied social positions that are traditionally associated with women, such as being a wife and mother. Despite her high-ranking positions in politics, she has faced systematic subordination and oppression due to her gender, such as sexism and gender bias, which satisfies condition (iii). Therefore, according to this definition, Hillary Clinton is a woman."
2,Pamela Anderson,True,"Pamela Anderson is regularly observed to have bodily features associated with the female biological role in reproduction. She is also marked within the dominant ideology of her society as someone who ought to occupy certain kinds of social positions that are often considered subordinate. Furthermore, these factors play a role in her systematic subordination in some dimensions of her social position. Therefore, according to the given definition, Pamela Anderson is a woman."
3,Lebron James,False,"Lebron James does not meet any of the criteria outlined in the definition of a woman. He is not observed or imagined to have the bodily features associated with a female's biological role in reproduction. He is not marked within the dominant ideology of his society as someone who ought to occupy certain kinds of social position that are subordinate. And his social position is not oppressive, nor does he satisfy criteria (i) and (ii) that would play a role in any such subordination. Therefore, according to this definition, Lebron James is not a woman."
4,Sally Haslanger,This cannot be accurately answered with 'True' or 'False' as it requires personal and societal context about Sally Haslanger that is not provided in the question.,"The definition provided for a woman is complex and involves multiple factors, including physical characteristics, societal expectations, and experiences of subordination. Without specific information about Sally Haslanger's physical characteristics, her societal context, and her personal experiences, it is impossible to definitively say whether or not she fits this definition. Therefore, a simple 'True' or 'False' answer would not be accurate or fair."
5,Nancy Pelosi,True,"Nancy Pelosi is observed to have the bodily features associated with a female's biological role in reproduction. She is also marked within the dominant ideology of her society as someone who ought to occupy certain kinds of social position. While it can be argued that her position as Speaker of the House is not subordinate, it is important to note that the definition does not require all women to occupy subordinate positions, but rather that they are marked as such within their society's dominant ideology. Therefore, Nancy Pelosi satisfies both conditions (i) and (ii) of the definition."
6,Catwoman,True,"Catwoman, also known as Selina Kyle, is a fictional character in the Batman series who is regularly and for the most part observed or imagined to have certain bodily features presumed to be evidence of a female’s biological role in reproduction. This marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate. Furthermore, the fact that she satisfies these conditions plays a role in her systematic subordination, that is, along some dimension, her social position is oppressive, and her satisfy